In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from lesview import *

In [ ]:
casenames = dict(
    c1 = 'R11_MSM97-CT_Stokes_f0_fixdt3',
    c2 = 'R11_MSM97-CT_Stokes_f0_fixdt3_visL',
    # c2 = 'R11_MSM97-CT_Stokes_f0_trs4',
    # c3 = 'R11_MSM97-CT_Stokes_f0',
    # c4 = 'R11_MSM97-CT_Stokes_f0_fixdt3_IE',
    # c5 = 'R11_MSM97-CT_Stokes_f0_trs4_IL2',
    # c6 = 'R11_MSM97-CT_Stokes_f0_r2_IL',
)
models = dict(
    c1 = 'ncarles',
    c2 = 'ncarles',
    # c2 = 'oceananigans',
    # c3 = 'oceananigans',
    # c4 = 'ncarles',
    # c5 = 'oceananigans',
    # c6 = 'oceananigans',
)
filenames = dict(
    ncarles = 'his.mp.vis.000001.028801.nc',
    oceananigans = 'averages.jld2',
)
snapshots = ['2000-01-01T02:00:00', '2000-01-01T03:00:00', '2000-01-01T06:00:00', '2000-01-01T12:00:00']
hours = [2, 3, 6, 12]
colors = ['tab:blue', 'tab:green', 'tab:orange', 'tab:red']
abc = 'abcdef'

figpath = 'R11_MSM97_CT_Stokes_f0'
os.makedirs(figpath, exist_ok=True)

In [ ]:
ds = {}
for cn in casenames.keys():
    print(cn)
    datapath = os.path.join(os.path.pardir, models[cn], casenames[cn], filenames[models[cn]])
    if models[cn] == 'ncarles':
        ds[cn] = NCARLESDataProfile(filepath=datapath).dataset
    elif models[cn] == 'oceananigans':
        ds[cn] = OceananigansDataProfile(filepath=datapath).dataset
    else:
        raise ValueError('Model {:s} not supported'.format(models[cn]))
        

In [ ]:
ds['c1']

In [ ]:
g = 9.81
a = 0.8
wavenum = 2*np.pi/60
f = np.sqrt(g*wavenum)
us0 = a**2*wavenum*f
alpha = 2e-4
NNT0 = 0.01
NN0 = alpha*g*NNT0
Qt0 = 1.221e-5
h0 = 33
wstar = (alpha*g*Qt0*h0)**(1/3)
depth = 50
def get_vars(ds, var):
    if 'uxym' in ds.data_vars.keys():
        if var == 'uL':
            return (ds.data_vars['uxym']+ds.data_vars['stokes'])/wstar
        elif var == 'uE':
            return ds.data_vars['uxym']/wstar
        elif var == 'uS':
            return ds.data_vars['stokes']/wstar
        elif var == 'NN':
            return ds.data_vars['txym'].differentiate(coord='z')/NNT0
        elif var == 'wNN':
            return -ds.data_vars['uxym'].differentiate(coord='z')*ds.data_vars['stokes'].differentiate(coord='z')/NN0
        elif var == 'tNN':
            return ds.data_vars['txym'].differentiate(coord='z')/NNT0-ds.data_vars['uxym'].differentiate(coord='z')*ds.data_vars['stokes'].differentiate(coord='z')/NN0
        elif var == 'wu':
            return ds.data_vars['uwle']
        elif var == 'wusb':
            return ds.data_vars['uwsb']
        else:
            raise ValueError('Variable {:s} not found.'.format(var))
    else:
        us = xr.zeros_like(ds.data_vars['u'])
        us.data = np.broadcast_to(us0 * np.exp(2.*wavenum*us.z), [us.time.size, us.z.size]).transpose()
        if var == 'uL':
            return ds.data_vars['u']/wstar
        elif var == 'uE':
            return (ds.data_vars['u'] - us)/wstar
        elif var == 'uS':
            return us/wstar
        elif var == 'NN':
            return ds.data_vars['T'].differentiate(coord='z')/NNT0
        elif var == 'wNN':
            return -(ds.data_vars['u']-us).differentiate(coord='z')*us.differentiate(coord='z')/NN0
        elif var == 'tNN':
            return ds.data_vars['T'].differentiate(coord='z')/NNT0-(ds.data_vars['u']-us).differentiate(coord='z')*us.differentiate(coord='z')/NN0
        elif var == 'wu':
            return ds.data_vars['wu']
        elif var == 'wusb':
            return ds.data_vars['wusb']
        else:
            raise ValueError('Variable {:s} not found.'.format(var))

In [ ]:
cn = 'c1'
fig, axarr = plt.subplots(1, 2, sharey='row')
fig.set_size_inches(5.5,2.8)

ax = axarr[0]
ax.axvline(x=0, color='k')
us = get_vars(ds[cn], 'uS').sel(time='2000-01-01T01:00:00', method='nearest')
us.plot(ax=ax, y='z', color='gray', linestyle='--')
(-us).plot(ax=ax, y='z', color='k', linestyle=':')
for j, ss in enumerate(snapshots):
    uL = get_vars(ds[cn], 'uL').sel(time=ss, method='nearest')
    uL.plot(ax=ax, y='z', color=colors[j])
    uLmean = uL.where(uL.z>=-depth).mean(dim='z')
    ax.plot(uLmean,0,'x',color=colors[j],clip_on=False,zorder=3)
    get_vars(ds[cn], 'uE').sel(time=ss, method='nearest').plot(ax=ax, y='z', color=colors[j], linestyle=':')
    ax.set_xlabel('$\overline{u}^L/w_*$, $\overline{u}/w_*$, $u^S/w_*$')
    ax.set_xlim([-7, 7])
ax.plot(np.nan, np.nan, color='k', label='$\overline{u}^L/w_*$')
ax.plot(np.nan, np.nan, color='k', linestyle=':', label='$\overline{u}/w_*$')
ax.plot(np.nan, np.nan, color='k', linestyle='--', label='$u^S/w_*$')
ax.legend(loc='lower left', framealpha=1)


ax = axarr[1]
nn = get_vars(ds[cn], 'NN').sel(time='2000-01-01T00:00:00', method='nearest')
nn.plot(ax=ax, y='z', color='k')
(nn + us.differentiate(coord='z')**2*wstar**2/NN0).plot(ax=ax, y='z', color='k', linestyle='--')
for j, ss in enumerate(snapshots):
    get_vars(ds[cn], 'tNN').sel(time=ss, method='nearest').plot(ax=ax, y='z', color=colors[j], linestyle='--')
    get_vars(ds[cn], 'NN').sel(time=ss, method='nearest').plot(ax=ax, y='z', color=colors[j])
ax.set_xlabel('$N^2/N^2_0$, $N_*^2/N^2_0$')
ax.set_xlim([-1, 3])

for i in np.arange(2):
    ax = axarr[i]
    ax.text(0.95, 0.05, '({:s})'.format(abc[i]), transform=ax.transAxes, va='bottom', ha='right')
    ax.grid()
    ax.set_title('')
    ax.set_ylabel('')
    ax.set_ylim([-depth,0])

ax.plot(np.nan, np.nan, color='k', label='$N^2/N^2_0$')
ax.plot(np.nan, np.nan, color='k', linestyle='--', label='$N_*^2/N^2_0$')
lg1 = ax.legend(loc='center right', framealpha=1)
    
axarr[0].set_ylabel('$z$ [m]')
  
lh = []
ltmp, = ax.plot(np.nan, np.nan, color='k', label='$t = 0$ hr')
lh.append(ltmp)
for j, _ in enumerate(snapshots):
    ltmp, = ax.plot(np.nan, np.nan, color=colors[j], label='$t = {:g}$ hr'.format(hours[j]))
    lh.append(ltmp)
lg2 = ax.legend(handles=lh, loc='center right', bbox_to_anchor=(0, 0.5), framealpha=1)

ax.add_artist(lg1)

# plt.tight_layout()
plt.subplots_adjust(top=0.97, bottom=0.18, left=0.12, right=0.97, hspace=0.3, wspace=0.15)
figname = os.path.join(figpath, 'profiles-ncar-trans')
fig.savefig(figname, dpi = 300, facecolor='w')

In [ ]:
cns = ['c1', 'c2']
xlabels = ['$\overline{u}^L/w_*$, $\overline{u}/w_*$, $u^S/w_*$', '$N^2/N^2_0$, $N_*^2/N^2_0$']
fig, axarr = plt.subplots(2, 2, sharey='row', sharex='col')
fig.set_size_inches(5.5,5.5)
for i, cn in enumerate(cns):
    ax = axarr[i,0]
    ax.axvline(x=0, color='k')
    us = get_vars(ds[cn], 'uS').sel(time='2000-01-01T01:00:00', method='nearest')
    us.plot(ax=ax, y='z', color='gray', linestyle='--')
    (-us).plot(ax=ax, y='z', color='k', linestyle=':')
    for j, ss in enumerate(snapshots):
        uL = get_vars(ds[cn], 'uL').sel(time=ss, method='nearest')
        uL.plot(ax=ax, y='z', color=colors[j])
        uLmean = uL.where(uL.z>=-depth).mean(dim='z')
        ax.plot(uLmean,0,'x',color=colors[j],clip_on=False,zorder=3)
        get_vars(ds[cn], 'uE').sel(time=ss, method='nearest').plot(ax=ax, y='z', color=colors[j], linestyle=':')
        ax.set_xlim([-7, 7])
    ax.plot(np.nan, np.nan, color='k', label='$\overline{u}^L/w_*$')
    ax.plot(np.nan, np.nan, color='k', linestyle=':', label='$\overline{u}/w_*$')
    ax.plot(np.nan, np.nan, color='k', linestyle='--', label='$u^S/w_*$')
    ax.legend(loc='lower left', framealpha=1)
    
    ax = axarr[i,1]
    nn = get_vars(ds[cn], 'NN').sel(time='2000-01-01T00:00:00', method='nearest')
    nn.plot(ax=ax, y='z', color='k')
    (nn + us.differentiate(coord='z')**2*wstar**2/NN0).plot(ax=ax, y='z', color='k', linestyle='--')
    for j, ss in enumerate(snapshots):
        get_vars(ds[cn], 'tNN').sel(time=ss, method='nearest').plot(ax=ax, y='z', color=colors[j], linestyle='--')
        get_vars(ds[cn], 'NN').sel(time=ss, method='nearest').plot(ax=ax, y='z', color=colors[j])
    ax.set_xlim([-1, 3])
    ax.plot(np.nan, np.nan, color='k', label='$N^2/N^2_0$')
    ax.plot(np.nan, np.nan, color='k', linestyle='--', label='$N_*^2/N^2_0$')
    if i == 0:
        lg1 = ax.legend(loc='center right', framealpha=1)
    else:
        ax.legend(loc='center right', framealpha=1)

for i in np.arange(2):
    for j in np.arange(2):
        ax = axarr[i,j]
        m = i*2+j
        ax.text(0.95, 0.05, '({:s})'.format(abc[m]), transform=ax.transAxes, va='bottom', ha='right')
        ax.grid()
        ax.set_title('')
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_ylim([-depth,0])

for i in np.arange(2):
    ax = axarr[i,0]
    ax.set_ylabel('$z$ [m]')
    ax = axarr[1,i]
    ax.set_xlabel(xlabels[i])

ax = axarr[0,1]
lh = []
ltmp, = ax.plot(np.nan, np.nan, color='k', label='$t = 0$ hr')
lh.append(ltmp)
for j, _ in enumerate(snapshots):
    ltmp, = ax.plot(np.nan, np.nan, color=colors[j], label='$t = {:g}$ hr'.format(hours[j]))
    lh.append(ltmp)
lg2 = ax.legend(handles=lh, loc='center right', bbox_to_anchor=(0, 0.5), framealpha=1)

ax.add_artist(lg1)

# plt.tight_layout()
plt.subplots_adjust(top=0.97, bottom=0.1, left=0.12, right=0.97, hspace=0.15, wspace=0.15)
figname = os.path.join(figpath, 'profiles-ncar-trans-4p')
fig.savefig(figname, dpi = 300, facecolor='w')